Detect Corner 

In [4]:
import cv2
import numpy as np

In [18]:
image_path= r"C:\Users\91744\Desktop\rbe 549\P1_starter_code\YourDirectoryID_p1\Phase1\Data\Train\Set1\1.jpg"


img= cv2.imread(image_path)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# img, blocksize, ksize, k
# This will give us intensity of detected corners so this is all that matters
probab_corner= cv2.cornerHarris(gray,7,11,0.04) 
print(probab_corner.max(), " ", probab_corner.min())
dst = cv2.dilate(probab_corner,None)
img[probab_corner>0.01*probab_corner.max()]=[0,0,255]

threshold= 0.01*probab_corner.max()
detected_corner_masks= probab_corner>threshold
detected_corner_masks= (detected_corner_masks*255).astype(np.uint8)
# probab_corner= cv2.normalize(probab_corner, None, 0,255,cv2.NORM_MINMAX)
# print(probab_corner.max(), " ", probab_corner.min())
print((detected_corner_masks[0,0]))
cv2.imshow("corners highlighted", img)

cv2.waitKey(0)
cv2.destroyAllWindows()

1915105.2   -444763.2
0


In [32]:
import matplotlib.pyplot as plt
from skimage.feature import peak_local_max
import math

In [ ]:
# We will now try to look for local maxima in probab_corner

# probab_corner
threshold= 0.01*probab_corner.max()
probab_corner= np.where(probab_corner>threshold, probab_corner, 0)
local_maximas= peak_local_max(probab_corner, min_distance=7)


# Checking stuff
# print(type(local_maximas))
# print(local_maximas)
# plt.imshow(probab_corner, cmap='gray')
# plt.plot(local_maximas[:, 1], local_maximas[:, 0], 'r.')
# plt.show()

# Ordering functions
def dist(coord1, coord2):
    return np.sqrt((coord1[0]-coord2[0])**2+(coord1[1]-coord2[1])**2)

# Ordering by distance
def calc_distances(coords):
    distances= {}
    total_elements=coords.size
    for i in range(total_elements):
        if i not in distances:
            distances[i]= math.inf
        for j in range(i+1, total_elements):
            if j not in distances:
                distances[j]= math.inf
            temp_dist= dist(coords[i], coords[j])
            if(temp_dist<distances[i]):
                distances[i]=temp_dist
            if(temp_dist<distances[j]):
                distances[j]=temp_dist
    return distances

def select_top(n, coords):
    distances= calc_distances(coords)
    top_n=[]
    distances= dict(sorted(distances.items(), key=lambda item: item[1], reverse=True))
    print(distances)
    print(distances.keys())
    for i in range(n):
        top_n.append()

select_top(10,local_maximas)

print(local_maximas.size)
print(type(local_maximas[0,0]))

TypeError: select_top() missing 1 required positional argument: 'coords'